In [36]:
# load in data

import pandas

formatted_data = pandas.read_csv('formatted_data.csv')
formatted_data.to_dict()
results = pandas.read_csv('results.csv')
results.to_csv()
total_data = len(results['Prediction'])

data_occupation_race = formatted_data['Ocuppation Race']
data_participant_race = formatted_data['Other Participant Race']
data_gender = formatted_data['Gender']

for i in range(total_data):
    if type(data_occupation_race[i]) != str:
        data_occupation_race[i] = ''
    if type(data_participant_race[i]) != str:
        data_participant_race[i] = ''

print(data_occupation_race)
print(data_participant_race)
result_predictions = results['Prediction']
result_true_values = results['True Value']
result_confidence = results['Confidence']

/var/folders/cq/jfz677p53hd170kx2kmc7plr0000gq/T/ipykernel_7948/645473616.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_occupation_race[i] = ''
/var/folders/cq/jfz677p53hd170kx2kmc7plr0000gq/T/ipykernel_7948/645473616.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_participant_race[i] = ''


0                  
1                  
2                  
3                  
4                  
            ...    
25915    Indigenous
25916    Indigenous
25917    Indigenous
25918    Indigenous
25919    Indigenous
Name: Ocuppation Race, Length: 25920, dtype: object
0                  
1                  
2                  
3                  
4                  
            ...    
25915    Indigenous
25916    Indigenous
25917    Indigenous
25918    Indigenous
25919    Indigenous
Name: Other Participant Race, Length: 25920, dtype: object


In [42]:

# import xrarray as xr

# analyzed_data = xr.Dataset({'True Occupation': dict(), 'False Occupation': dict(), 'True Participant': dict(), 'False Participant': dict()})

# analyzed_data = {'True Occupation': dict(), 'False Occupation': dict(), 'True Participant': dict(), 'False Participant': dict()}
keys = ['True Occupation', 'False Occupation', 'True Participant', 'False Participant']
analyzed_data = dict()
for race1 in races:
    analyzed_data[race1] = dict()
    for race2 in races:
        analyzed_data[race1][race2] = dict()
        for gender in genders:
            analyzed_data[race1][race2][gender] = dict()
            for key in keys:
                analyzed_data[race1][race2][gender][key] = 0
                

# print(result_predictions[0])
# print(data_occupation_race)
# print(data_participant_race)

for i in range(total_data):
    # print(data_occupation_race[i], data_participant_race[i], data_gender[i])
    occupation_race = data_occupation_race[i]
    if result_confidence[i] < 0.8:
        continue
    if result_predictions[i] == 'participant':
        if str(result_true_values[i]) == 'participant':
            analyzed_data[data_occupation_race[i]][data_participant_race[i]][data_gender[i]]['True Participant'] += 1
        else:
            analyzed_data[data_occupation_race[i]][data_participant_race[i]][data_gender[i]]['False Participant'] += 1
    else:
        if result_true_values[i] == 'occupation':
            analyzed_data[data_occupation_race[i]][data_participant_race[i]][data_gender[i]]['True Occupation'] += 1
        else:
            analyzed_data[data_occupation_race[i]][data_participant_race[i]][data_gender[i]]['False Occupation'] += 1


rates = list(analyzed_data.keys())
for i in range(len(rates)):
    rates[i] = f'{rates[i]} Rate'


# tpr = tp / (tp + fn)

def find_rates(rate, actual_val, all_vals):
    for race1 in races:
        for race2 in races:
            for gender in genders:
                analyzed_data[race1][race2][gender][rate] = analyzed_data[race1][race2][gender][actual_val] / (analyzed_data[race1][race2][gender][actual_val] + analyzed_data[race1][race2][gender][all_vals])

def find_rates_list(rate, numerators, denominators):
    # gets accuracy
    total_numerator = 0
    total_denominator = 0
    for race1 in races:
        for race2 in races:
            for gender in genders:
                numerator = 0
                denominator = 0
                for n in numerators:
                    numerator += analyzed_data[race1][race2][gender][n]
                for d in denominators:
                    denominator += analyzed_data[race1][race2][gender][d]
                total_numerator += numerator
                total_denominator += denominator
                analyzed_data[race1][race2][gender][rate] = numerator / denominator

    print("Total Accuracy:", total_numerator / total_denominator)

find_rates('True Participant Rate', 'True Participant', 'False Occupation')
find_rates('False Participant Rate', 'False Participant','True Occupation')
find_rates('True Occupation Rate', 'True Occupation', 'False Participant')
find_rates('False Occupation Rate', 'False Occupation', 'True Participant')
find_rates_list('Accuracy', ['True Participant', 'True Occupation'], ['True Participant', 'True Occupation', 'False Participant', 'False Occupation'])

# # print(analyzed_data)
# pd.DataFrame.from_dict({(i,j): user_dict[i][j] 
#                            for i in user_dict.keys() 
#                            for j in user_dict[i].keys()},
#                        orient='index')

formatted_dict = dict()
for race1 in races:
    for race2 in races:
        for gender in genders:
            formatted_dict[(race1, race2, gender)] = analyzed_data[race1][race2][gender]


data_frame = pandas.DataFrame.from_dict(formatted_dict, orient='index')
# data_frame = pandas.DataFrame.from_dict({(rate, race1, race2): analyzed_data[rate][race1][race2]
#                                         for rate in list(analyzed_data.keys())
#                                         for race1 in list(analyzed_data[rate].keys())
#                                         for race2 in list(analyzed_data[rate][race1].keys())},
#                                     orient='index')

# data_frame = pandas.DataFrame.from_dict(analyzed_data)
# data_frame = pandas.DataFrame(analyzed_data)
data_frame.to_csv('analyzed_data.csv')


Total Accuracy: 0.47540983606557374
